# Agentic Framework in Llama Index

## Tool Use

The FunctionTool module can be used here to convert any function into a tool that the agent could call if it feels the necessity



In [1]:
import requests
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

from llama_index.llms.openai import OpenAI
import nest_asyncio
from llama_index.agent.introspective import IntrospectiveAgentWorker
from llama_index.agent.introspective import (
    ToolInteractiveReflectionAgentWorker,
)
import os

import dotenv


from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

nest_asyncio.apply()

dotenv.load_dotenv()

True

In [2]:
# Load API Keys
API_KEY = os.getenv('OPENWEATHER_API_KEY')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### 1. To access external API data 

In [8]:
def get_weather(city_name, units='imperial'):
    """
    Function that taken in a city name and units and returns the temperature and description of weather

    Args:
    city_name: The name of the city (ex: Los Angeles)
    units: The measurement units for tempreature (Options: imperial - fahreinheit, metric - celsius), Default: fahrenheit (imperial)

    Return:
    city: Name of the City
    temperature: the temperature in the provided measurement units
    description: the short description of the weather
    """
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + API_KEY + "&q=" + city_name + "&units=" + units
    
    response = requests.get(complete_url)
    
    if response.status_code == 200:
        weather_data = response.json()
        return {
            'city': city_name,
            'temperature': weather_data['main']['temp'],
            'description': weather_data['weather'][0]['description']
        }
    else:
        return {'error': 'Failed to retrieve weather data', 'status_code': response.status_code}

In [9]:
# Declare tool
weather_fetch_tool = FunctionTool.from_defaults(fn=get_weather)

#### Instance of simple function call using the OpenAI LLM

In [10]:
# Use query engine with the tool

llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)
response = llm.predict_and_call(
    [weather_fetch_tool], 
    "How is the weather in San Jose right now?", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: get_weather with args: {"city_name": "San Jose"}
=== Function Output ===
{'city': 'San Jose', 'temperature': 67.57, 'description': 'clear sky'}
{'city': 'San Jose', 'temperature': 67.57, 'description': 'clear sky'}


#### Using the agent

### Combination of Runner and Worker

Runner - Manages/Orchestrates the different tasks once the query is received, oversees the outputs of the worker, returns final response when ready

Worker - Does the actual execution of the individual tasks using the available tools

![components](/Users/chandrasekaransidha/personal/sandbox/img/agent_components.png)

In [11]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [weather_fetch_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)


In [12]:
response = agent.query("Who is Connor McGregor?")

Added user message to memory: Who is Connor McGregor?
=== LLM Response ===
Conor McGregor is a professional mixed martial artist and former UFC (Ultimate Fighting Championship) featherweight and lightweight champion. He is known for his charismatic personality, trash-talking, and fighting skills inside the octagon. McGregor is one of the most popular and successful fighters in the history of MMA (Mixed Martial Arts).


In [13]:
response = agent.query("Is it a good day to go running outside, I'm in Hayward?")

Added user message to memory: Is it a good day to go running outside, I'm in Hayward?
=== Calling Function ===
Calling function: get_weather with args: {"city_name": "Hayward"}
=== Function Output ===
{'city': 'Hayward', 'temperature': 61.36, 'description': 'scattered clouds'}
=== LLM Response ===
The current temperature in Hayward is 61.36°F with scattered clouds. It seems like a good day to go running outside!


In [14]:
# Using the ReAct Agent

agent = ReActAgent.from_tools([weather_fetch_tool], llm=llm, verbose=True)


response = agent.chat("Is it a good day to go running outside, I'm in Hayward?")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_weather
Action Input: {'city_name': 'Hayward', 'units': 'imperial'}
Observation: {'city': 'Hayward', 'temperature': 61.36, 'description': 'scattered clouds'}
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The current temperature in Hayward is 61.36°F with scattered clouds. It seems like a decent day to go running outside.


### 2. Auto Retrieval: To perform metadata filtering while retrieving data (RAG) 

## Autogen

In [35]:
!pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 19.5 MB/s eta 0:00:00


In [15]:
import autogen

In [20]:
llm_config = {"model": "gpt-4o"}

In [17]:
task = '''
        Write a concise but engaging blogpost about
       the cultural effect of Garfield, the fictional character created by Jim Davis. 
       Make sure the blogpost is within 100 words.
       '''

In [21]:
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [22]:
reply = writer.generate_reply(
        messages=[{"content": task, "role": "user"}]
        )

print(reply)

**The Cultural Impact of Garfield: A Lasagna-Loving Icon**

Since his debut in 1978, Jim Davis's Garfield has transcended his comic-strip origins to become a global phenomenon. This lasagna-loving, Monday-hating cat has left an indelible mark on pop culture, inspiring TV shows, merchandise, and even philosophical musings on apathy and indulgence. Garfield's relatable disdain for Mondays and love for comfort has resonated across generations, making him a timeless symbol of modern life's everyday struggles. Beyond humor, Garfield offers a mirror to society's quirks, proving that even a simple comic strip can spark widespread cultural dialogues.


### Reflection using Autogen

In [23]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [24]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       the cultural effect of Garfield, the fictional character created by Jim Davis. 
       Make sure the blogpost is within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**The Cultural Impact of Garfield: A Lasagna-Loving Icon**

Since his debut in 1978, Jim Davis's Garfield has transcended his comic-strip origins to become a global phenomenon. This lasagna-loving, Monday-hating cat has left an indelible mark on pop culture, inspiring TV shows, merchandise, and even philosophical musings on apathy and indulgence. Garfield's relatable disdain for Mondays and love for comfort has resonated across generations, making him a timeless symbol of modern life's everyday struggles. Beyond humor, Garfield offers a mirror to society's quirks, proving that even a simple comic strip can spark widespread cultural dialogues.

-------------------------

### Nested Chat for getting multiple perspectives

In [26]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [25]:
Format_reviewer = autogen.AssistantAgent(
    name="Format Reviewer",
    llm_config=llm_config,
    system_message="You are a Text Format reviewer, known for "
        "your ability to make a block of text appealing when viewed as markdown, "
        "you ensure that the text is easy and pleasant to read for a viewer," 
        "broken down into digestible paragraphs and headings and punctuations represented"
        "in a way that renders accurately in a markdown format"
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [32]:
Casualness_reviewer = autogen.AssistantAgent(
    name="Casualness Reviewer",
    llm_config=llm_config,
    system_message="You are a Text Casualness reviewer, known for "
        "your ability to review a piece of text and identify phrases that seem too boring and complex"
        "and propose ways to make them more catchy and relatable"
        "to make the piece engaging at the same time retaining the quality of the message"
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [33]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [34]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, # The input message is an llm summarization of the task message
     "summary_method": "reflection_with_llm", # The output provided is also refined by an llm
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",}, # Mentioned only once, the other reviewers follow suit
     "max_turns": 1},
    {
    "recipient": Format_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": Casualness_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [35]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [36]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       the cultural effect of Garfield, the fictional character created by Jim Davis. 
       Make sure the blogpost is within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**The Cultural Impact of Garfield: A Lasagna-Loving Icon**

Since his debut in 1978, Jim Davis's Garfield has transcended his comic-strip origins to become a global phenomenon. This lasagna-loving, Monday-hating cat has left an indelible mark on pop culture, inspiring TV shows, merchandise, and even philosophical musings on apathy and indulgence. Garfield's relatable disdain for Mondays and love for comfort has resonated across generations, making him a timeless symbol of modern life's everyday struggles. Beyond humor, Garfield offers a mirror to society's quirks, proving that even a simple comic strip can spark widespread cultural dialogues.

-------------------------

**Notes:**
- One reviwers comments passed on to the next, so the order matters - as the earlier reviewers suggestions would impact the review of the other
- Need to maintain a good pull between the different perspectives to ensure quality of the final message (ex: Casualness vs Appeal vs Quality)